## TJ's Item Nutrition Visualization

#### Load HP Store Data

In [1]:
import polars as pl
tjhp_clean = pl.read_csv("data/tjhp_clean.csv")

### Hyde Park Viz

In [7]:
tjhp_clean.columns

['sku',
 'name',
 'calories_per_dollar',
 'dollars_per_calorie',
 'price_usd',
 'calories_per_container',
 'calories_per_serving',
 'servings_per_container',
 'dollars_per_serving']

In [8]:
import altair as alt

# alt.Chart(tjhp_clean).mark_point().encode(
#     x='price_usd',
#     y='calories_per_container',
#     color='calories_per_serving',
#     tooltip=['sku', 'name']
# ).interactive()

# https://github.com/vega/altair/issues/1245
# How to disable automatic alphabetic sorting of x value
alt.Chart(tjhp_clean.sort(by='dollars_per_calorie', descending=False).head(20)).mark_bar().encode(
    # x='name',
    x=alt.X('name', title='Product', type='nominal', sort=None, axis=alt.Axis(labelAngle=-45)),
    y=alt.Y(field='dollars_per_100calorie', type='quantitative', title='$ per 100 Calories'),
    color=alt.Color('dollars_per_serving', title='$ per Serving').scale(scheme="greens"),
    tooltip=['sku', 'name', 'price_usd', 'calories_per_container', 'dollars_per_calorie', 'servings_per_container']
).transform_calculate(
    dollars_per_100calorie='datum.dollars_per_calorie * 100' # Calculate the new field
).interactive(

).properties(
    title='Hyde Park TJ\'s 20 items with the cheapest calories',
    width=800,
    height=300
).configure_title(
    fontSize=24,
    # font='IMPACT',
    anchor='middle',
    color='black'
).configure_axis(
    labelFontSize=10,
    titleFontSize=20
)

# tjhp_clean.columns

alt.Chart(...)

## Viz

In [ ]:
import altair as alt

In [ ]:

alt.Chart(tj_clean).mark_point().encode(
    x='price_usd',
    y='calories_per_container',
    color='calories_per_serving',
    tooltip=['sku', 'name']
).interactive()

In [ ]:
# https://github.com/vega/altair/issues/1245
# How to disable automatic alphabetic sorting of x value
alt.Chart(tj_clean.sort(by='calories_per_dollar', descending=True).head(20)).mark_bar().encode(
    # x='name',
    x=alt.X('name', type='nominal', sort=None, axis=alt.Axis(labelAngle=-45)),
    y='calories_per_dollar',
    color=alt.Color('dollars_per_serving').scale(scheme="greens"),
    tooltip=['sku', 'name', 'price_usd', 'calories_per_serving', 'spc']
).interactive(

).properties(
    width=800,
    height=300
)

In [ ]:
tj_clean.columns

## Misc

In [ ]:
import pandas as pd

In [ ]:
ptj = pd.DataFrame(tj)

In [ ]:
ptj.columns = tj.columns

In [ ]:
ptj.head(3)

In [ ]:
ptj['nutrition'][0][0]

In [ ]:
ptj

In [ ]:
# ptj['nutrition'].apply(lambda x: x[0] if len(x)>0 else None)

In [ ]:
# tj['nutrition'].struct.field('calories_per_serving')

In [ ]:
# tj.with_columns(
#     pl.col("nutrition").map_elements(lambda x: x[0][0]).struct.field('calories_per_serving').alias("calories_per_serving")
# )

In [ ]:
# temp = tj.with_columns(
#     pl.col("nutrition").struct.field("calories_per_serving").alias("calories_per_serving")
# )
# temp